Creating an image classification application using Vision Transformers (ViT), and then containerizing it with Docker and deploying it to Kubernetes involves several steps. Below, I’ll outline the entire process, including code snippets and commands.

### Step 1: Create a Simple Image Classification Code with ViT

First, we need to create a Python script that performs image classification using a Vision Transformer model. For this example, we'll use the `transformers` library from Hugging Face and `torch`.

#### Create a File Structure

```
image_classifier/
│
├── app.py
├── requirements.txt
├── Dockerfile
└── k8s-deployment.yaml
```

#### `app.py`

Here’s a simple image classification application using ViT:

```python
# app.py
import torch
from torchvision import transforms
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor

# Load the pre-trained ViT model and processor
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

# Function to predict the class of an image
def classify_image(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        logits = model(**inputs).logits
    
    predicted_class_idx = logits.argmax(-1).item()
    return predicted_class_idx

if __name__ == "__main__":
    import sys
    if len(sys.argv) != 2:
        print("Usage: python app.py <image_path>")
        sys.exit(1)
    image_path = sys.argv[1]
    class_idx = classify_image(image_path)
    print(f"Predicted class index: {class_idx}")
```

#### `requirements.txt`

List the required libraries:

```
torch==1.12.1
transformers==4.20.1
Pillow==9.1.0
```

### Step 2: Create a Dockerfile

Next, we create a Dockerfile to containerize our application.

#### `Dockerfile`

```dockerfile
# Dockerfile
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY app.py .

ENTRYPOINT ["python", "app.py"]
```

### Step 3: Build the Docker Image

Navigate to the `image_classifier` directory and build the Docker image:

```bash
docker build -t image-classifier .
```

### Step 4: Run the Docker Container

You can test the Docker container by running it with an image:

```bash
docker run --rm -v $(pwd):/app image-classifier /app/sample_image.jpg
```

Make sure to replace `sample_image.jpg` with the path to an actual image you want to classify.

### Step 5: Create Kubernetes Deployment

Now, we will create a Kubernetes deployment to run our container.

#### `k8s-deployment.yaml`

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: image-classifier
spec:
  replicas: 2
  selector:
    matchLabels:
      app: image-classifier
  template:
    metadata:
      labels:
        app: image-classifier
    spec:
      containers:
      - name: image-classifier
        image: image-classifier:latest
        command: ["python", "app.py", "/path/to/image.jpg"]
```

### Step 6: Deploy to Kubernetes

Before deploying, ensure that your Docker image is pushed to a container registry that your Kubernetes cluster can access (e.g., Docker Hub, Google Container Registry).

1. **Tag and Push the Image**:

```bash
docker tag image-classifier your-dockerhub-username/image-classifier:latest
docker push your-dockerhub-username/image-classifier:latest
```

2. **Deploy to Kubernetes**:

```bash
kubectl apply -f k8s-deployment.yaml
```

### Step 7: Accessing Your Application

Once the deployment is running, you can expose it via a service. Here’s an example of a service YAML:

```yaml
apiVersion: v1
kind: Service
metadata:
  name: image-classifier
spec:
  selector:
    app: image-classifier
  ports:
  - protocol: TCP
    port: 80
    targetPort: 8080
  type: LoadBalancer
```

Apply the service:

```bash
kubectl apply -f k8s-service.yaml
```

### Step 8: Using the Container

You can send requests to your service endpoint (LoadBalancer IP or NodePort) to classify images. You might need to adapt the code to expose an API endpoint (like Flask or FastAPI) to accept images and return predictions.

### Summary

1. **Developed** a simple image classification script using ViT.
2. **Containerized** the application using Docker.
3. **Deployed** the Docker container to Kubernetes.
4. **Exposed** the application via a Kubernetes service for external access.

Feel free to modify the code and deployment configurations to better suit your use case!